# 自编码器
## TODO https://www.zhihu.com/question/41490383

### 自编码器简介
&emsp;&emsp;先不谈神经网络、深度学习、仅是自编器的的话，其原理很简单。自编码器可以理解为一个试图去还原始输入的系统。如下图所示：

<img src="f5ccfacef17d59e106b012af8676d3b5_hd.jpg" width = "60%"/>

&emsp;&emsp;图中，虚线蓝色框内就是一个自编码器模型，它由编码器（Encoder）和解码器（Decoder）两部分组成，本质上都是对输入信号做某种变换。编码器将输入信号$x$变换成编码信号$y$，然后解码器将编码$y$转化成输出信息$\widetilde{x}$。即
$$y = f(x)\\ \widetilde{x} = g(y) = g(f(x))$$

&emsp;&emsp;自编码器的目的是，让输出$\widetilde{x}$尽可能复现输入$x$，即“Tries to copy its input to its output”。但是如果一个自编码器只是简单地学会$g(f(x)) = x$，也就是恒有$x = \widetilde{x}$，那么这个编码器并没有什么特别的用处（并无卵用）。相反，我们不应该将自编码器设计成输入到输出完全相等。这通常需要向自编码器加一些约束，是它只能近似地复制，并只能复制与训练数据相似的输入。这些约束强制模型考虑输入数据的哪些部分需要被优秀复制，因此它往往能学习到数据的有用特性。

&emsp;&emsp;对于自编码器，我们往往并不关心输出是什么（反正只是复现输入）,我们真正关心的是中间层的编码，或者说是从输入到编码的映射。可以这么想，在我们强迫编码$y$和输入$x$不同的情况下，系统还能够去复原原始信号$x$,那么说明编码$y$已经承载了原始数据的所有信息，但以一种不同的形式 —— 这就是特征提取，而且是自动学出来的。实际上，自动学习原始特征表达也是神经网络和深度学习的核心目的之一。

## 自编码与神经网络